# IMDB XGBoost Revenue
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Importing so that I can use LinearRegression and OLS Models.
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics

In [2]:
final_df = pd.read_csv('../data/final_df.csv')

In [3]:
final_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

### XGBoost

In [6]:
from xgboost import XGBRegressor

In [7]:
xgb = XGBRegressor()

In [441]:
features = ['duration', 'Adventure',  'Biography', 'Crime', 'Drama', 'Fantasy',  'Horror', 'Sci-Fi', 'Thriller', 'avg_vote', 
            'us_voters_votes', 'votes', 'votes_1', 'votes_3', 'votes_7', 'votes_8', 'votes_9',  'popularity', 
            'director_score', 'actor_score', 'plot_sentiment', 'total_score', 'profitable', 'budget']

In [442]:
X = final_df[features]
y = final_df['revenue']

In [443]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [444]:
xgb.fit(X_train, y_train);

In [446]:
predictions = xgb.predict(X_train)

residuals = y_train - predictions

rss = (residuals ** 2).sum()

mse = metrics.mean_squared_error(y_train, predictions)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))

In [447]:
print("RMSE:", rmse)
print(" ")
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

RMSE: 3484400.334059546
 
Training Score: 0.9996707037918515
 
Testing Score: 0.631778115156665


In [448]:
null_predictions = np.zeros_like(y_test)

null_pred = null_predictions + y_test.mean()

baseline = metrics.mean_squared_error(y_test, null_pred, squared = False)

In [449]:
print("Baseline RMSE:", baseline)
print(" ")
print("RMSE:", rmse)
print("________________________________")
print(" ")
print("Difference:", baseline - rmse)

Baseline RMSE: 155540661.6705855
 
RMSE: 3484400.334059546
________________________________
 
Difference: 152056261.33652598


## GridSearch

In [450]:
xgb = XGBRegressor()

In [451]:
model_params = {}
count = 0

In [455]:
xgb_params = {
    'learning_rate': [.05, .10, .15, .20],
    'max_depth' : [1, 2, 3, 4],
    'n_estimators': [50, 75, 100, 200, 300],
    'subsample' : [1]
}

gs = GridSearchCV(xgb, param_grid = xgb_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient = 'index')

model_df

,learning_rate,max_depth,n_estimators,subsample,best_score,training_score,testing_score
model_1,0.1,3,100,1,0.668441,0.919870,0.549541
model_2,0.1,2,100,1,0.690756,0.869525,0.539882
model_3,0.1,2,100,1,0.690756,0.869525,0.539882
model_4,0.1,2,100,1,0.690756,0.869525,0.539882
